In [1]:
import pandas as pd
import numpy as np

## Randomly selecting 12 papers from 150 Good research paper

In [2]:
# Read in the csv file
## This is a csv file that teammates manually inspected as good research pape
random_papers_df = pd.read_csv("150 Research paper_small_classification.csv")['Link']
random_papers_df.head()

0    https://www.science.org/doi/10.1126/science.ad...
1    https://www.nature.com/articles/s41566-019-0398-2
2    https://www.nature.com/articles/s41560-020-007...
3    https://www.science.org/doi/10.1126/science.ab...
4    https://www.nature.com/articles/s41467-021-236...
Name: Link, dtype: object

In [3]:
sampled_good = random_papers_df.sample(n=12, random_state=1)  # random_state for reproducibility
sampled_good

14              https://doi.org/10.1126%2Fsciadv.abj7930
98           https://doi.org/10.1038%2Fs41560-019-0538-4
75                  https://doi.org/10.1039%2FC9TA12489F
16                  https://doi.org/10.1039%2FD0TA10535J
131         https://doi.org/10.1038%2Fs41586-023-06208-z
56              https://doi.org/10.1002%2Faenm.202101447
141         https://doi.org/10.1038%2Fs41586-023-06637-w
44     https://www.sciencedirect.com/science/article/...
29                  https://doi.org/10.1039%2FC8TC04871A
120    https://www.science.org/doi/10.1126/science.ab...
94     https://onlinelibrary.wiley.com/doi/10.1002/ad...
5      https://www.sciencedirect.com/science/article/...
Name: Link, dtype: object

In [4]:
## Exporting this as a csv file name "good_paper_small"
sample_df_good = sampled_good.to_frame(name='link')
sample_df_good['label'] = 1
sample_df_good.to_csv('good_paper_small.csv', index=False)

## Randomly selecting 12 papers from BAD research Paper

We will randomly select 12 and manually inspect if these paper are actually "bad"

In [5]:
# Read in the csv file
random_papers_df = pd.read_csv("bad_paper_big.csv")
random_papers_df.head()

,link,doi
0,https://doi.org/10.1038%2Fnature14133,10.1038%2Fnature14133
1,https://doi.org/10.1038%2Fnmat4014,10.1038%2Fnmat4014
2,https://doi.org/10.1039%2FC5EE03874J,10.1039%2FC5EE03874J
3,https://doi.org/10.1038%2Fs41563-018-0071-z,10.1038%2Fs41563-018-0071-z
4,https://doi.org/10.1002%2Faenm.201700491,10.1002%2Faenm.201700491


In [6]:
sampled_bad_df = random_papers_df.sample(n=12, random_state=1)  # random_state for reproducibility
sampled_bad_df

,link,doi
24,https://doi.org/10.1002%2Fadma.201607039,10.1002%2Fadma.201607039
39,https://doi.org/10.1039%2FC7TA00434F,10.1039%2FC7TA00434F
52,https://doi.org/10.1002%2Faenm.201601079,10.1002%2Faenm.201601079
27,https://doi.org/10.1038%2Fs41467-017-00516-5,10.1038%2Fs41467-017-00516-5
44,https://doi.org/10.1021%2Facsenergylett.7b00236,10.1021%2Facsenergylett.7b00236
2,https://doi.org/10.1039%2FC5EE03874J,10.1039%2FC5EE03874J
21,https://doi.org/10.1126%2Fsciadv.aao5616,10.1126%2Fsciadv.aao5616
62,https://doi.org/10.1103%2FPhysRevLett.77.3865,10.1103%2FPhysRevLett.77.3865
41,https://doi.org/10.1002%2Faenm.201701136,10.1002%2Faenm.201701136
50,https://doi.org/10.1557%2Fmrc.2015.26,10.1557%2Fmrc.2015.26


In [7]:
sampled_bad_df['label'] = 0
sampled_bad_df.to_csv('bad_paper_small.csv', index=False)

In [8]:
sampled_bad_df

,link,doi,label
24,https://doi.org/10.1002%2Fadma.201607039,10.1002%2Fadma.201607039,0
39,https://doi.org/10.1039%2FC7TA00434F,10.1039%2FC7TA00434F,0
52,https://doi.org/10.1002%2Faenm.201601079,10.1002%2Faenm.201601079,0
27,https://doi.org/10.1038%2Fs41467-017-00516-5,10.1038%2Fs41467-017-00516-5,0
44,https://doi.org/10.1021%2Facsenergylett.7b00236,10.1021%2Facsenergylett.7b00236,0
2,https://doi.org/10.1039%2FC5EE03874J,10.1039%2FC5EE03874J,0
21,https://doi.org/10.1126%2Fsciadv.aao5616,10.1126%2Fsciadv.aao5616,0
62,https://doi.org/10.1103%2FPhysRevLett.77.3865,10.1103%2FPhysRevLett.77.3865,0
41,https://doi.org/10.1002%2Faenm.201701136,10.1002%2Faenm.201701136,0
50,https://doi.org/10.1557%2Fmrc.2015.26,10.1557%2Fmrc.2015.26,0
